# Korzystam z https://pokeapi.co/

## Evolution_stage_map - nazwa pokemona i numer ewolucji

In [3]:
import requests
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
#..PLIK EVOLUTION_STAGE_MAP.CSV

#..rekurencja wyciągające dane z łańcucha
def extract_stages(chain_link, stage, results):
    pokemon_name = chain_link['species']['name']
    results.append({
        'name': pokemon_name.capitalize(),
        'Evolution_Stage': stage
    })
    
    if chain_link['evolves_to']:
        for next_link in chain_link['evolves_to']:
            extract_stages(next_link, stage + 1, results)

#..pobranie listy URL do wszystkich łańcuchów, potem ich szczegóły
def fetch_all_evolution_chains():
    all_results = []
    
    #..pobranie wszyskich dostępnych łańcuchów - limit 2000 
    list_url = "https://pokeapi.co/api/v2/evolution-chain/?limit=2000"
    #..ładne wyświetlanie się w terminalu
    print("Pobieranie listy łańcuchów...")

    try:
        list_response = requests.get(list_url, timeout=10)
        list_response.raise_for_status()
        chain_urls = [result['url'] for result in list_response.json()['results']]
    except Exception as e: #..obsługa błędów
        print(f"Błąd przy pobraniu: {e}")
        return pd.DataFrame()

    #..iteracja po konkretnych URL
    for url in tqdm(chain_urls, desc="Pobieranie szczegółów ewolucji"):
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            data = response.json()
            
            extract_stages(data['chain'], 1, all_results)
            time.sleep(0.05) #..pauza

        except Exception as e:
            print(f"\nBłąd dla URL {url}: {e}")
            continue
    #..zwracanie dataframe        
    return pd.DataFrame(all_results)

#..URUCHOMIENIE PROGRAMU
df_evolution_map = fetch_all_evolution_chains()

#..sortowanie alfabetyczne
df_evolution_map = df_evolution_map.sort_values(by='name')

output_filename = 'evolution_stage_map.csv'
df_evolution_map.to_csv(output_filename, index=False)

print("\n-------------------------------------------------")
print(f"Liczba pobranych wierszy: {len(df_evolution_map)}")
print(f"Zapisano do: {output_filename}")

Pobieranie listy łańcuchów...


Pobieranie szczegółów ewolucji:   6%|▌         | 32/541 [00:04<01:14,  6.81it/s]

In [ ]:
#..PLIK EVOLUTION_PATHS.CSV

#..rekurencja budująca ścieżki ewolucji
def build_evolution_paths(chain_link, current_path, all_paths):
    pokemon_name = chain_link['species']['name'].capitalize()
    new_path = current_path + [pokemon_name]

    if not chain_link['evolves_to']:
        all_paths.append(new_path)
    else:
        for next_link in chain_link['evolves_to']:
            build_evolution_paths(next_link, new_path, all_paths)

#..główna logika - pobranie listy wszystkich dostępnych łańcuchów, potem budowanie ścieżek.
def fetch_all_paths_dynamic():
    
    all_raw_paths = []
    
    #..pobranie listy URL do wszystkich łańcuchów, znowu limit 2000
    list_url = "https://pokeapi.co/api/v2/evolution-chain/?limit=2000"
    print("Pobieranie listy łańcuchów...")

    try:
        list_resp = requests.get(list_url, timeout=10)
        list_resp.raise_for_status()
        chain_urls = [r['url'] for r in list_resp.json()['results']]
    except Exception as e:
        print(f"Błąd pobierania listy: {e}")
        return pd.DataFrame()

    #..iteracja po konkretnych URL
    for url in tqdm(chain_urls, desc="Przetwarzanie ścieżek"):
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            data = response.json()
            
            build_evolution_paths(data['chain'], [], all_raw_paths)
            time.sleep(0.05) 
            
        except Exception as e:
            print(f"Błąd dla {url}: {e}")
            continue

    #..tworzenie dataframe
    df = pd.DataFrame(all_raw_paths)
    
    #..zależy nam tylko na podstawowych ewolucjach - trzy kolumny
    if df.shape[1] > 3:
        df = df.iloc[:, :3]

    df.columns = [f'Stage {i+1}' for i in range(df.shape[1])]
    return df

#..URUCHOMIENIE PROGRAMU
df_paths = fetch_all_paths_dynamic()

output_filename = 'evolution_paths.csv'
df_paths.to_csv(output_filename, index=False)

print(f"\nZapisano plik: {output_filename}")
print(f"Liczba wierszy (ścieżek): {len(df_paths)}")
print(df_paths.head())

Pobieranie listy łańcuchów...


Przetwarzanie ścieżek: 100%|██████████| 541/541 [01:31<00:00,  5.89it/s]


Zapisano plik: evolution_paths_ALL.csv
Liczba wierszy (ścieżek): 568
      Stage 1     Stage 2     Stage 3
0   Bulbasaur     Ivysaur    Venusaur
1  Charmander  Charmeleon   Charizard
2    Squirtle   Wartortle   Blastoise
3    Caterpie     Metapod  Butterfree
4      Weedle      Kakuna    Beedrill
